In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
import sys
module = os.path.abspath('C:/Users\\thinkpad\\OneDrive\\Data-Science\\Workstress\\Mods')
if module not in sys.path:
    sys.path.append(module)
from DataManager import DataManager
from Demo import Demo

In [ ]:
manager = DataManager()

In [ ]:
manager.load_all()

In [ ]:
manager.compute_features();
manager.compute_features_stress();

In [ ]:
print("We have", len(manager.SUBJECTS), " subjects")

for feature in manager.FEATURES.keys():
    print("there are ", len(manager.FEATURES[feature]), " values for ", feature)
#     print(manager.FEATURES[feature][3277:3288])

lets reshape data such that there are [samples, features].

In [ ]:
# samples, features = [N, 15]

X1 = []
X2 = []
for i in range(0,  len(manager.FEATURES['a_mean'])):
    X1.append([manager.FEATURES['a_mean'][i], manager.FEATURES['a_std'][i], manager.FEATURES['a_maxx'][i], manager.FEATURES['a_maxy'][i], manager.FEATURES['a_maxz'][i],\
                  manager.FEATURES['e_max'][i], manager.FEATURES['e_min'][i], manager.FEATURES['e_mean'][i], manager.FEATURES['e_range'][i], manager.FEATURES['e_std'][i],\
                  manager.FEATURES['t_max'][i], manager.FEATURES['t_min'][i], manager.FEATURES['t_mean'][i], manager.FEATURES['t_range'][i], manager.FEATURES['t_std'][i]])
print(np.shape(X1))

for i in range(0,  len(manager.STRESS_FEATURES['a_mean'])):
    X2.append([manager.STRESS_FEATURES['a_mean'][i], manager.STRESS_FEATURES['a_std'][i], manager.STRESS_FEATURES['a_maxx'][i], manager.STRESS_FEATURES['a_maxy'][i], manager.STRESS_FEATURES['a_maxz'][i],\
                  manager.STRESS_FEATURES['e_max'][i], manager.STRESS_FEATURES['e_min'][i], manager.STRESS_FEATURES['e_mean'][i], manager.STRESS_FEATURES['e_range'][i], manager.STRESS_FEATURES['e_std'][i],\
                  manager.STRESS_FEATURES['t_max'][i], manager.STRESS_FEATURES['t_min'][i], manager.STRESS_FEATURES['t_mean'][i], manager.STRESS_FEATURES['t_range'][i], manager.STRESS_FEATURES['t_std'][i]] )
print(np.shape(X2))

In [ ]:
# print(manager.FEATURES['a_mean'][100:110])
# print(manager.FEATURES['a_std'][400:410])
# print(manager.FEATURES['a_maxx'][0:10])

In [ ]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import keras
from keras.preprocessing import sequence

from keras.models import Sequential

from keras.layers import Dense, Embedding, Flatten

from keras.layers import LSTM

In [ ]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

# session_conf = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8, )

# tf.set_random_seed(1)

# session = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# keras.backend.set_session(session)

## Initialize two arrays for the targets.

In [ ]:
# x1 is the baseline data
# x2 is the stress data

y1 = [0] * len(X1)
y2 = [1] * len(X2)

In [ ]:
print(len(y1))
print(len(y2))

Lets concatenate the data between baseline and stress so that we can split it into training and test sets

In [ ]:
X = np.concatenate((X1, X2), axis=0)
print(np.shape(X))

y = np.concatenate((y1,y2), axis=0)
print(np.shape(y))

## Splitting the data up in train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25, random_state=42)

Scaling the data so we can get better results.

   1. fit the scaler to the training data (fit_transform)
   2. transform training data with the scaler
   3. fit the model with transformed data
   4. transform test data with the scaler
   5. predict using model and output of (4)

It will be better to have our data scaled between 0 and 1, lets normalize the data that way now.

In [ ]:
def normalize(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)
X_train = normalize(X_train)
X_test = normalize(X_test)

The input_shape should be (number of sequences=?, time_steps=None, features=15) and the target should be (number of sequences, time_steps, targets). Since our data already has implicit timesteps from the feature calculations, we will set time_steps = 1.

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

In [ ]:
num_neurons = 15
num_features = 15

In [ ]:
print('[-]Building the LSTM network...')

model = Sequential()
model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=True))
# model.add(LSTM(16, input_shape=(1, 15), dropout=0.35, recurrent_dropout=0.35, return_sequences=True))
model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=False))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.summary()

print("inputs: " , model.input_shape)
print("outputs: ", model.output_shape)
print("actual inputs: ", np.shape(X_train))
print("actual outputs: ", np.shape(y_train))

In [ ]:
from keras.optimizers import SGD
opt = SGD(lr=0.05)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
print('Training LSTM...')

batch_size = 2 # I think I want to use batch_size = 1

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)

In [ ]:
print('score:', score)
print('accuracy:', acc)

In [ ]:
#Saving the model to disc
json = model.to_json()
with open("model.json", "w") as file:
    file.write(json)
model.save_weights("model.h5")
print("Saved model to disc")

In [ ]:
from keras.models import model_from_json
# Load the model of interest
json_file = open('model.json', 'r')
json = json_file.read()
json_file.close()
model_from_disc = model_from_json(json)
model_from_disc.load_weights("model.h5")

In [ ]:
opt = SGD(lr=0.05)
model_from_disc.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

score, acc = model_from_disc.evaluate(X_test , y_test, batch_size=batch_size)
print('score:', score)
print('accuracy:', acc)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_pred = model_from_disc.predict(X_test, batch_size=2, verbose=1)

In [ ]:
print(y_pred)

In [ ]:
y_pred[y_pred>0.5] = 1 
y_pred[y_pred<=0.5] = 0 

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

The LSTM-based network architecture with a single hidden layer has achieved a validation accuracy of around 97.7% after 5 epochs, which is quite impressive. Even with just one epoch, the model has shown validation accuracy results ranging from 80% to 92%. Training each epoch takes about 70 seconds without GPU acceleration. Moreover, with less modalities and features, the model surpasses the WESAD quoted results for accuracy and F1 score after just 5 epochs.